<a href="https://colab.research.google.com/github/co-min/ai_basic_study/blob/main/job_sct_sample_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 진로문장완성검사 Open API 데이터셋 활용하여 파인튜닝하기

과정


```
로컬 데이터 업로드 → 전처리 → LLaMA LoRA 파인튜닝 → 테스트
```


# 1. 데이터셋 참고하기

[진로문장완성검사 텍스트 데이터](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&searchKeyword=%EC%A7%84%EB%A1%9C%EB%AC%B8%EC%9E%A5%EC%99%84%EC%84%B1%EA%B2%80%EC%82%AC%20%ED%85%8D%EC%8A%A4%ED%8A%B8%20%EB%8D%B0%EC%9D%B4%ED%84%B0&aihubDataSe=data&dataSetSn=71791)

In [ ]:
!pip install -U transformers
!pip install -U datasets
!pip install -U peft
!pip install -U accelerate
!pip install -U bitsandbytes

# Hugging Face Transformers: 모델 로드/학습
# PEFT: LoRA 기반 경량 파인튜닝
# Datasets: 데이터셋 처리
# Accelerate: 학습 최적화
# BitsAndBytes: VRAM을 줄이면서 모델을 학습할 수 있도록 4bit 양자화 지원

# 2. 데이터 업로드

Inference(추론)은 AI가 이미 학습한 지식을 바탕으로 빠르게 정답을 추론하거나 응답을 생성하는 과정입니다. 예를 들어, 챗봇(Chatbot)에게 "오늘 날씨 어때? "라고 물으면, 모델은 자신이 학습한 데이터에서 가장 적절한 답을 빠르게 찾아 제공

---

AI 학습에서는 일반적으로 입력 → 출력 관계를 학습합니다.

여기서는 원천 데이터를 instruction, 라벨 데이터를 output 컬럼으로 통일합니다.

.head()로 데이터가 잘 합쳐졌는지 확인합니다.

**개념 설명**
---

**glob에서 recursive=True 의미**

recursive=True는 주로 프로그래밍에서 사용되는 옵션으로, 어떤 작업이나 함수가 자기 자신을 반복적으로 호출하여 작업을 수행하는 "재귀적인" 방식을 사용할 것인지를 나타냄. 특히 파일 시스템 탐색과 같이 계층적인 구조를 다룰 때, 하위 폴더까지 포함하여 검색 범위를 확장할 때 사용됨.

In [ ]:
# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. 필요 라이브러리 임포트
import os
import glob
import json
import pandas as pd



# 3. 원천/라벨링 폴더 경로
source_folder = "/content/drive/MyDrive/dataset_job_sample/01_sorce_data"
label_folder  = "/content/drive/MyDrive/dataset_job_sample/02_labeling_data"

# 4. JSON 파일 경로를 재귀적으로 찾기
source_files = sorted(glob.glob(os.path.join(source_folder, "**", "*.json"), recursive=True))
label_files  = sorted(glob.glob(os.path.join(label_folder, "**", "*.json"), recursive=True))

print(f"원천 데이터 파일 수: {len(source_files)}")
print(f"라벨링 데이터 파일 수: {len(label_files)}")

# 파일 수가 동일한지 확인
assert len(source_files) == len(label_files), "원천 데이터와 라벨링 데이터 파일 수가 일치하지 않습니다."


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
원천 데이터 파일 수: 293
라벨링 데이터 파일 수: 293


In [ ]:
# 5. JSON 읽어서 DataFrame 생성


instruction_list = [] # 명령어나 질문을 모아두는 리스트
output_list = [] # 각 명령어에 대한 결과값을 저장하는 리스트

# 5-1. 각 JSON 파일을 열어 질문과 답변 추출
for src_path, lbl_path in zip(source_files, label_files):
    with open(src_path, 'r', encoding='utf-8') as f_src, open(lbl_path, 'r', encoding='utf-8') as f_lbl:
        # 파일 안의 JSON 데이터를 파이썬 딕셔너리(key-value)로 변환
        src_data = json.load(f_src)
        lbl_data = json.load(f_lbl)


        # JSON 구조에 맞게 키 수정 필요
        question = src_data["test"][0]["answer_full"].strip()
        answer   = lbl_data["comment"]["comment"].strip()

        instruction_list.append(question)
        output_list.append(answer)



# 5-2. instruction/output 형태로 합치기
dataset_df = pd.DataFrame({
    "instruction": instruction_list,
    "output": output_list
})

# 5-3. 샘플 확인
dataset_df.head()

,instruction,output
0,"내가 가장 좋아하는 활동은 반려동물과 놀기, 연락하기이다","반려동물과 놀기, 연락하는 활동이 좋은 이유에 대해서 탐색하고 단순히 휴식이나 놀이..."
1,나는 많은 사람과 함께 있을 때 시끄러워서 선호하지 않는다,자신의 대인관계 성향을 이해하고 있다. 내향적이며 다수의 사람들과의 관계 형성에 불...
2,친구들과 사이좋게 지내려면 나는 말을 잘 섞어보려 노력한다,긍정적인 또래관계를 형성하는데 필요한 관계역량을 알고 있으나 구체적으로 표현하지 못...
3,내가 하고 싶은 직업을 갖기 위해 나는 한국사를 열심히 한다,자신의 진로목표와 연계하여 구체적인 노력을 기울이고 있다.
4,가장 기억에 남는 직업체험활동은 웹 드라마 촬영이다 그 이유는 처음으로 내 진로에 ...,자신의 관심분야와 연관성이 높은 직업체험활동을 함으로 직업에 대한 관심을 직접적인 ...


# 3. Hugging Face Dataset 변환

학습용 데이터와 검증용 데이터로 나누어, 모델이 학습 중 얼마나 잘 맞추는지 평가

검증용 데이터는 모델 학습에 직접 쓰지 않음

In [ ]:
from datasets import Dataset

# Pandas DataFrame → Hugging Face Dataset 변환
hf_dataset = Dataset.from_pandas(dataset_df)

# 90% 학습용, 10% 검증용 분리
# hf_dataset = datasets.train_test_split(test_size=0.1)


# 4. 모델 & 토크나이저 및 데이터셋 전처리

토크나이징: 텍스트를 토큰으로 바꾸는 과정

1. Instruction + Output + EOS → 모델 입력
2. input_ids = 모델 입력, labels = 정답
3. .map으로 데이터셋 전체를 한 번에 처리


def tokenize_fn(example)
---
데이터셋에서 한 개의 샘플을 받아서, 모델이 학습할 수 있는 토큰 형태로 바꿔주는 함수



```
prompt = example["instruction"]
학습 데이터에서 모델에게 지시할 내용(Instruction)을 꺼냅니다.

target = example["output"]
모델이 답해야 할 정답(Output)을 꺼냅니다.
```



```
full_input = f"{prompt}\n{target}{tokenizer.eos_token}"
Instruction과 Output을 합쳐서 모델 입력으로 만듭니다.

\n → 줄바꿈, 보기 좋게 분리
tokenizer.eos_token → 문장의 끝을 알려주는 특별 토큰(EOS) 추가
```



```
tokenized = tokenizer(full_input, truncation=True, padding="max_length", max_length=256)

1. 모델이 이해할 수 있는 숫자(토큰)로 변환
2. truncation=True → 256 토큰 이상이면 자름
3. padding="max_length" → 256 토큰보다 짧으면 0으로 채움
4. max_length=256 → 최대 길이 256 토큰
```



```
tokenized["labels"] = tokenized["input_ids"]

1. 모델이 학습할 정답(Label) 지정
2. Auto-regressive 모델에서는 입력과 정답이 같아야 학습 가능

return tokenized
-> 토큰화된 딕셔너리를 반환
예)

{
  "input_ids": [101, 2054, 2003, 102, ...],
  "attention_mask": [1, 1, 1, 1, ...],
  "labels": [101, 2054, 2003, 102, ...]
}

```



```
# tokenized_dataset = hf_dataset.map(tokenize_fn, batched=False)


1. hf_dataset = Hugging Face 데이터셋
2. .map(tokenize_fn) → 모든 샘플에 tokenize_fn 적용
3. batched=False → 한 번에 1개 샘플씩 처리

결과: tokenized_dataset에는 모델이 바로 학습할 수 있는 형태의 데이터가 들어갑니다.

```













In [ ]:
from transformers import AutoTokenizer

# 테스트용 작은 LLaMA 모델
# model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model_name = "meta-llama/Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# instruction/output → 모델 입력 형태로 변환
# 토크나이저 및 데이터셋 전처리
def tokenize_fn(example):
    prompt = example["instruction"]
    target = example["output"]
    # Use a structured prompt template with a system instruction
    system_instruction = "너는 진로 상담을 해주는 전문 상담가야."
    full_input = f"{system_instruction}\n\n### Instruction:\n{prompt}\n\n### Response:\n{target}{tokenizer.eos_token}"
    tokenized = tokenizer(full_input, truncation=True, padding="max_length", max_length=256)
    tokenized["labels"] = tokenized["input_ids"] # Add labels
    return tokenized

tokenized_dataset = hf_dataset.map(tokenize_fn, batched=False)

Map:   0%|          | 0/293 [00:00<?, ? examples/s]

# 5. 모델 불러오기 및 LoRA 설정

LoRA: 모델 전체를 학습하지 않고 일부만 학습 → Colab에서도 가능

4bit 양자화: VRAM을 1/4로 줄여 큰 모델도 학습 가능

In [ ]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

# 모델 불러오기 (4bit 양자화 + 자동 디바이스 할당)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.bfloat16 # Use bfloat16 for potential compatibility with 4-bit and GPU
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)


# LoRA 설정 (VRAM 절약용, 모델 일부만 학습)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head"],  # Include more modules for better performance
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

trainable params: 22,030,336 || all params: 8,052,291,584 || trainable%: 0.2736


# 6. 학습 실행

작은 테스트 학습이므로 max_steps=50

실제 학습 시 num_train_epochs와 배치 사이즈 조정

fp16=True로 VRAM 절약

---

[wandb.ai 토큰 url 바로가기](https://wandb.ai/niobium41next42-duksung-women-s-university?shareProfileType=copy)

참고 블로그


[1. WanB weight&bias](https://blog.naver.com/jungs-note/222844839062)

WandB는 개발자들이 머신러닝 모델을 더 효율적으로 구현할 수 있도록 제공되는 머신러닝 플랫폼

[2. wandb 사용법](https://changsroad.tistory.com/481)




1. TrainingArguments 설정

```
training_args = TrainingArguments(
    output_dir="./llama-lora-test",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=10,
    max_steps=50,
    fp16=True,
    logging_steps=10
)
```
- output_dir: 학습 후 모델 체크포인트, 로그, 최종 모델이 저장되는 경로입니다.
- per_device_train_batch_size: 각 GPU(또는 CPU)에서 처리하는 배치 크기입니다. VRAM이 작으면 줄여야 합니다.
- gradient_accumulation_steps: 작은 배치로 여러 step의 gradient를 쌓아 실제 큰 배치 크기 효과를 내는 옵션입니다. VRAM 절약 목적.
- learning_rate: 학습률, 0.0002로 설정되어 있습니다.
- num_train_epochs: 전체 데이터셋을 몇 번 반복해서 학습할지 지정합니다. 여기서는 테스트용으로 10번.
- max_steps: 최대 학습 step 수를 제한합니다. num_train_epochs보다 우선합니다. 여기서는 50 step만 학습합니다.
- fp16: Mixed Precision 학습 사용. VRAM 절약과 속도 향상 목적.
- logging_steps: 몇 step마다 로그를 기록할지 지정.

- 주석 처리된 save_steps, save_total_limit
  - 체크포인트 저장 관련 옵션입니다.


2. Trainer 객체 생성


```
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)
```
model: 학습할 모델 객체입니다. (예: LLaMA, GPT 등)

args: 위에서 정의한 TrainingArguments 객체.

train_dataset: 학습 데이터셋. tokenized_dataset은 tokenizer를 적용한 데이터여야 합니다.

주석 처리된 부분:

  - eval_dataset: 검증 데이터셋 지정. 여기서는 생략됨.
  - tokenizer: 모델에 맞는 tokenizer 지정 가능.




3. 학습 실행

```
trainer.train()
```

- Trainer가 실제 학습을 시작합니다.
- 지정된 max_steps 혹은 num_train_epochs 조건에 따라 학습 종료.
- 로그는 logging_steps 간격으로 출력됩니다.
- 학습 후 모델은 output_dir에 저장됩니다.



In [ ]:
from transformers import TrainingArguments, Trainer

# 학습 설정
training_args = TrainingArguments(
    output_dir="./llama-lora-test",
    per_device_train_batch_size=2,  # GPU 메모리 적은 경우 조정
    gradient_accumulation_steps=2,  # 배치 쌓아서 VRAM 절약
    learning_rate=2e-4,
    num_train_epochs=10,  # 샘플 테스트용 10 epoch
    max_steps=50,        # 샘플 학습용 step
    fp16=True,
    # save_steps=50,
    # save_total_limit=2,
    logging_steps=10
)

# Trainer 객체 생성 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset


    # train_dataset=tokenized_dataset["train"],
    # eval_dataset=tokenized_dataset["test"],
    # tokenizer=tokenizer
)

trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

# 7. 학습 완료 모델 저장

1. 모델 저장


```
model_save_path = "/content/drive/MyDrive/dataset_job_sample/llama_lora_trained"
model.save_pretrained(model_save_path)
```

- model.save_pretrained(path)
  - Hugging Face Transformers 모델을 지정한 경로에 저장합니다.
  - 저장되는 내용:
    1. pytorch_model.bin (모델 가중치)
    2. config.json (모델 설정)

- Google Colab 환경에서는 /content/drive/... 경로를 사용하면 드라이브에 직접 저장 가능합니다.



2. 토크나이저 저장


```
tokenizer.save_pretrained(model_save_path)
```

- tokenizer.save_pretrained(model_save_path)
토크나이저 관련 파일 저장:
  1. tokenizer.json (토큰화 규칙)
  2. vocab.txt 또는 merges.txt (모델 유형에 따라 다름)

- 모델과 함께 같은 디렉토리에 저장하면 나중에 불러오기 쉽습니다.


In [ ]:
model_save_path = "/content/drive/MyDrive/dataset_job_sample/llama_lora_trained"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"학습 완료 모델 저장 경로: {model_save_path}")

# 8. 학습된 모델 테스트

학습된 모델이 입력에 대해 적절한 출력을 생성하는지 확인
- strip() 함수는 문자열의 시작과 끝에서 공백을 제거한 후 반환


---
**요약**



1.   LoRA 학습 완료 모델과 토크나이저를 Google Drive에서 로드
2. VRAM 효율 위해 8bit 모델 로딩
3. pipeline으로 쉽게 텍스트 생성 설정
4. 샘플링 기반으로 자연스러운 답변 생성
5. 질문 부분 제거 후 모델 답변만 추출 가능






1️⃣ 모델과 토크나이저 로드


```
tokenizer = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModelForCausalLM.from_pretrained(model_save_path, device_map="auto", load_in_8bit=True)

```



- AutoTokenizer.from_pretrained(model_save_path)

  - 이전에 저장한 토크나이저를 불러옵니다.
  - 토크나이저가 학습 데이터와 동일하게 로드되어야 모델 입력이 일관됩니다.

- AutoModelForCausalLM.from_pretrained
  - 학습 완료 모델을 불러옵니다.
  - device_map="auto": GPU가 있으면 자동으로 GPU로 할당, 여러 GPU도 자동 분배
  - load_in_8bit=True: 모델을 8비트로 로드 → VRAM 절약, 추론 속도 향상

- 8bit 로딩은 LoRA나 LLM 추론에서 VRAM 최적화용으로 자주 사용됩니다.

2️⃣ 텍스트 생성 파이프라인 구성



```
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256,
    temperature=0.7,
    do_sample=True
)
```

- pipeline("text-generation")
  - Hugging Face가 제공하는 추론 파이프라인으로, 모델을 쉽게 텍스트 생성용으로 래핑합니다.
- max_length=256
  - 생성될 최대 토큰 길이

- temperature=0.7
  - 텍스트 다양성 조절. 낮으면 결정적, 높으면 다양성 증가

- do_sample=True
  - 샘플링 기반 생성. False면 greedy 방식(가장 높은 확률 토큰만 선택)




3️⃣ 테스트 질문 및 추론



```
test_question = "나는 모르는 직업이 있으면 찾아보지를 않는다."
result = generator(test_question)
generated_text = result[0]["generated_text"]
```

- generator(test_question) → 모델이 질문을 기반으로 텍스트 생성
- 결과는 리스트 안 딕셔너리 형태로 반환되며, generated_text 키에서 실제 문장 추출




4️⃣ 질문 제거 후 답변만 추출



```
answer_start_index = generated_text.find(test_question) + len(test_question)
model_answer = generated_text[answer_start_index:].strip()

```

- 생성 텍스트에서 질문 문장을 제거하고 모델이 실제 생성한 답변만 가져옵니다.
- strip()으로 앞뒤 공백 제거



5️⃣ 출력


```
print("질문:", test_question)
print("모델 답변:", generated_text)
```

- 질문과 모델이 생성한 전체 텍스트 출력
- model_answer를 출력하면 질문 제외 후 순수 답변만 확인 가능



In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# LoRA 학습 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModelForCausalLM.from_pretrained(model_save_path, device_map="auto", load_in_8bit=True)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256, # Increased max_length
    temperature=0.4,
    do_sample=True
)

# 테스트 질문
test_question = "나는 모르는 직업이 있으면 찾아보지를 않는다."
result = generator(test_question)
generated_text = result[0]["generated_text"]

# Remove the test_question from the generated_text
answer_start_index = generated_text.find(test_question) + len(test_question)
model_answer = generated_text[answer_start_index:].strip()

print("질문:", test_question)
print("모델 답변:", generated_text)

# 9. pipeline보다 세밀한 생성 제어 가능한, 프롬프트 기반으로 모델에 직접 토큰 입력 후 텍스트를 생성

1. 프롬프트를 정의하고 토크나이저로 텐서 변환
2. 모델 디바이스(GPU/CPU)에 입력
3. model.generate로 새 텍스트 생성
4. 토크나이저로 디코딩하여 출력

1️⃣ 프롬프트 정의



```
prompt = "### Instruction:\n서울은 한국의 수도입니까?\n\n### Response:\n"

```
모델에 입력할 질문과 지침을 문자열로 정의합니다.

"### Instruction:" / "### Response:" 포맷은 Instruction-tuning된 모델에서 자주 사용되는 구조입니다.

모델은 "Response" 이후 텍스트를 생성하게 됩니다.




2️⃣ 토큰화 및 장치 할당



```
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

```
tokenizer(prompt, return_tensors="pt") → 문자열을 PyTorch 텐서로 변환

.to(model.device) → GPU 또는 CPU에 텐서를 올림

이렇게 해야 모델이 입력 텐서를 읽을 수 있습니다.




3️⃣ 텍스트 생성



```
outputs = model.generate(**inputs, max_new_tokens=50)

```
model.generate → 직접 시퀀스 생성 수행

max_new_tokens=50 → 새로 생성될 토큰 수 제한

추가 옵션 없이 호출하면 기본 greedy 방식으로 생성됨 (가장 확률 높은 토큰 순서대로 선택)

- 더 자연스럽게 생성하려면 temperature, do_sample, top_k, top_p 등을 지정할 수 있습니다.


4️⃣ 생성 결과 디코딩



```
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

```

outputs[0] → 배치 첫 번째 결과 시퀀스

tokenizer.decode(..., skip_special_tokens=True) → 토큰을 사람이 읽을 수 있는 문자열로 변환

skip_special_tokens=True → <pad>, <eos> 같은 특수 토큰 제거



In [ ]:
# prompt = "### Instruction:\n서울은 한국의 수도입니까?\n\n### Response:\n"
# inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# outputs = model.generate(**inputs, max_new_tokens=50)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))